##### Importación de librerías

In [3]:
#Se importan las librerias a utilizar

import numpy as np
import pandas as pd
import os

----

##### Lectura de los datasets

In [4]:
urbansound8k_path = "../../data/raw/UrbanSound8K/"

# Primer dataset - Carpeta con los audios
urbansound8k_audio_path = "audio/"

# Segundo dataset - Archivo CSV con metadatos sobre los audios
urbansound8k_metadata_path_file = "metadata/UrbanSound8K.csv"

# Dataset CSV con metadatos generados a partir de los audios 
dataset_urbansound8k_path_file = "../../data/processed/cd02_nuevos_metadatos_v2.csv"

In [5]:
dataset_urbansound8k_df = pd.read_csv(urbansound8k_path + urbansound8k_metadata_path_file, sep=",")
dataset_urbansound8k_df.head(2)

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos 
4. Formateo definitivo


----

##### Selección de columnas

In [6]:
# Se establece qué columnas se eliminan
columnas_eliminar_lista = ['fsID', 'start', 'end', 'class']

# Se ejecuta la operación
dataset_urbansound8k_df.drop(columnas_eliminar_lista, inplace=True, axis=1) 

In [7]:
print("Vista del dataset de datos:")
display(dataset_urbansound8k_df.head(1))

Vista del dataset de datos:


,slice_file_name,salience,fold,classID
0,100032-3-0-0.wav,1,5,3


In [8]:
print("Columnas del dataset:")
print(dataset_urbansound8k_df.columns)
    
print(f"Cantidad de filas: {dataset_urbansound8k_df.shape[0]}")

Columnas del dataset:
Index(['slice_file_name', 'salience', 'fold', 'classID'], dtype='object')
Cantidad de filas: 8732


##### Filtrado de filas

In [9]:
# salience
temp_filtro_salience = dataset_urbansound8k_df[dataset_urbansound8k_df['salience'] != 1]

print(f"Campo 'salience' ")
print(f"cantidad filas afectadas por filtro: {temp_filtro_salience.shape[0]}")

Campo 'salience' 
cantidad filas afectadas por filtro: 3030


In [7]:
#duracion
Q1 = dataset_urbansound8k_df['duracion'].quantile(0.25)
Q3 = dataset_urbansound8k_df['duracion'].quantile(0.75)

RIQ = Q3 - Q1

# Definir los límites para identificar outliers
lower_bound = Q1 - 1.5 * RIQ
upper_bound = Q3 + 1.5 * RIQ

print("Q1 duracion ", Q1," y Q3 duracion ", Q3)

print("Limite inferior de duracion ", lower_bound)
print("Limite superior de duracion ", upper_bound)

# Filtrar los outliers
temp_filtro_duracion = dataset_urbansound8k_df[(dataset_urbansound8k_df['duracion'] < lower_bound) | (dataset_urbansound8k_df['duracion'] > upper_bound)]

# Ver los audios identificados como outliers
print(f"\nCampo 'duracion' ")
print(f"cantidad filas afectadas por filtro: {temp_filtro_duracion.shape[0]}")

Q1 duracion  4.0  y Q3 duracion  4.0
Limite inferior de duracion  4.0
Limite superior de duracion  4.0

Campo 'duracion' 
cantidad filas afectadas por filtro: 1407


In [8]:
# tasa_muestreo 
Q1 = dataset_urbansound8k_df['tasa_muestreo'].quantile(0.25)
Q3 = dataset_urbansound8k_df['tasa_muestreo'].quantile(0.75)
RIQ = Q3 - Q1

# Definir los límites para identificar outliers
lower_bound = Q1 - 1.5 * RIQ
upper_bound = Q3 + 1.5 * RIQ

print("Q1 tasa_muestreo ", Q1," y Q3 tasa_muestreo ", Q3)
print("Limite inferior de tasa_muestreo ", lower_bound)
print("Limite superior de tasa_muestreo ", upper_bound)

# Filtrar los outliers
temp_filtro_tasa_muestreo = dataset_urbansound8k_df[(dataset_urbansound8k_df['tasa_muestreo'] < lower_bound) | (dataset_urbansound8k_df['tasa_muestreo'] > upper_bound)]

# Ver los audios identificados como outliers
print(f"\nCampo 'tasa_muestreo' ")
print(f"cantidad filas afectadas por filtro: {temp_filtro_tasa_muestreo.shape[0]}")

Q1 tasa_muestreo  44100.0  y Q3 tasa_muestreo  48000.0
Limite inferior de tasa_muestreo  38250.0
Limite superior de tasa_muestreo  53850.0

Campo 'tasa_muestreo' 
cantidad filas afectadas por filtro: 860


In [9]:
# desplazamiento_dc
# Realizar una comprobación aproximada de desplazamiento de corriente continua (DC offset)
temp_filtro_desplazamiento_dc = dataset_urbansound8k_df[~(dataset_urbansound8k_df['desplazamiento_dc'] < 0.2)]

print(f"Campo 'desplazamiento_dc' ")
print(f"cantidad filas afectadas por filtro: {temp_filtro_desplazamiento_dc.shape[0]}")

Campo 'desplazamiento_dc' 
cantidad filas afectadas por filtro: 0


salience', 'classID', 'path', 'duracion', 'tasa_muestreo',
       'desplazamiento_dc', 'cantidad_canales', 'rango_dinamico', 'energia',
       'tasa_cruce_cero', 'entropia_espectograma', 'avg_centroide_espectral'],
      dtype='object'

In [10]:
# cantidad_canales 
temp_filtro_cantidad_canales = dataset_urbansound8k_df[~(dataset_urbansound8k_df['cantidad_canales'] == 1)]

print(f"Campo 'cantidad_canales' ")
print(f"cantidad filas afectadas por filtro: {temp_filtro_cantidad_canales.shape[0]}")

Campo 'cantidad_canales' 
cantidad filas afectadas por filtro: 7993


##### Construcción de atributos

Extraccion caracteristicas mfcc

In [11]:
import librosa
import librosa.display

def extract_mfcc(file_path, n_mfcc=13):
    signal, sr = librosa.load(file_path, sr=None, mono=True)  # Cargar el archivo de audio
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=n_mfcc)  # Extraer MFCC
    mfccs = np.mean(mfccs.T, axis=0)  # Tomar el promedio de los coeficientes MFCC en el tiempo
    return mfccs

In [12]:
# Crear columnas para cada uno de los 13 coeficientes MFCC
mfcc_columns = [f'mfcc_{i+1}' for i in range(13)]
mfcc_features = []

# Extraer MFCCs para todos los archivos
for index, row in dataset_urbansound8k_df.iterrows():
    mfcc = extract_mfcc(row['path'])
    mfcc_features.append(mfcc)

# Convertir la lista de MFCCs en un DataFrame y unirlo al DataFrame original
mfcc_df = pd.DataFrame(mfcc_features, columns=mfcc_columns)
dataset_urbansound8k_add_atributos_df = pd.concat([dataset_urbansound8k_df, mfcc_df], axis=1)

c:\Users\colbe\Documents\VIU\TFM\tfm-clasificador-fuentes-sonoras-ciudad\.env\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


In [13]:
dataset_urbansound8k_df.tail(3)

,salience,fold,classID,path,duracion,tasa_muestreo,desplazamiento_dc,cantidad_canales,rango_dinamico,energia,tasa_cruce_cero,entropia_espectograma,avg_centroide_espectral
8729,2,7,1,../../data/raw/UrbanSound8K//audio/fold7/99812...,3.505986,44100,0.000023,2,0.601532,152.930480,20199,9.236240,2588.530015
8730,2,7,1,../../data/raw/UrbanSound8K//audio/fold7/99812...,2.530000,44100,0.000023,2,0.155273,35.239840,19047,9.352435,2391.283507
8731,2,7,1,../../data/raw/UrbanSound8K//audio/fold7/99812...,2.530000,44100,0.000027,2,0.159119,47.776505,21736,9.474744,3079.384978


##### Formateo Definitivo

In [14]:
# Agregar en este listado otros atributos que pudieran discretizarse o transformarse
columnas_seleccionadas_final = [
              'classID',
              'tasa_cruce_cero',
              'entropia_espectograma',
              'avg_centroide_espectral',
              'fold',
              'mfcc_1', 'mfcc_2', 'mfcc_3',  'mfcc_4', 'mfcc_5',
              'mfcc_6', 'mfcc_7', 'mfcc_8',  'mfcc_9', 'mfcc_10',
              'mfcc_11', 'mfcc_12', 'mfcc_13']

# Crear un nuevo DataFrame con solo las columnas seleccionadas
dataset_urbansound8k_formato_final_df = dataset_urbansound8k_add_atributos_df[columnas_seleccionadas_final]

In [15]:
print(f"Cantidad de columnas del dataset final: {dataset_urbansound8k_formato_final_df.shape[1]}")
print(f"Cantidad de filas del dataset final: {dataset_urbansound8k_formato_final_df.shape[0]}")

Cantidad de columnas del dataset final: 18
Cantidad de filas del dataset final: 8732


Exportación del dataset

In [16]:
dataset_urbansound8k_formato_final_df.to_csv("../../data/final/dataset_urbansound8k_vfinal.csv", sep=';', index=False)